In [1]:
import json
import pandas as pd
import redis

# 連接Redis
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# 獲取最新的k線數據
latest_kline = r.get('latest_kline')

# 如果有數據的話
if latest_kline:
    kline_data = json.loads(latest_kline)
    print("最新的K線數據:")
    print(kline_data)

最新的K線數據:
{'symbol': 'SPOT_BTC_USDT', 'type': '5m', 'open': 96182.7, 'close': 96189.26, 'high': 96200.1, 'low': 96182.7, 'volume': 0.509165, 'amount': 48981.0157846, 'startTime': 1733313600000, 'endTime': 1733313900000}


In [ ]:
import asyncio
from redis import asyncio as aioredis
import pandas as pd
from io import StringIO

async def monitor_atr_and_price():
    """簡單的監控函數，只顯示當前 ATR 和價格"""
    try:
        # 連接到本地的 Redis 服務器
        redis_client = await aioredis.from_url("redis://localhost:6379")
        
        while True:
            # 從 Redis 獲取處理後的 K 線數據
            processed_klines = await redis_client.get('processed_klines')
            if processed_klines:
                # 將數據轉換為 DataFrame
                klines_str = processed_klines.decode('utf-8') if isinstance(processed_klines, bytes) else processed_klines
                df = pd.read_json(StringIO(klines_str))
                
                if not df.empty:
                    # 獲取最新的 ATR 和價格
                    current_atr = df['atr'].iloc[-1]
                    current_price = df['close'].iloc[-1]
                    
                    print(f"\nATR: {current_atr:.4f}")
                    print(f"當前價格: {current_price:.2f}")
            
            # 每 300 秒更新一次數據
            await asyncio.sleep(300)
            
    except Exception as e:
        print(f"監控出錯: {str(e)}")
    finally:
        await redis_client.close()

# 在 Jupyter Notebook 中運行這個監控函數
await monitor_atr_and_price()


ATR: 70.3632
當前價格: 96615.00


In [13]:
import json
import pandas as pd
import redis

# 連接Redis
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# 獲取處理後的DataFrame
processed_data = r.get('processed_klines')
if processed_data:
    df = pd.read_json(processed_data)
    display(df)

/var/folders/57/bvn1r6kj60v_d2chpsvv11480000gn/T/ipykernel_33524/3041318129.py:11: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(processed_data)


,timestamp,open,high,low,close,volume,atr,direction,tops,bottoms,signal
0,2024-12-05 03:42:00,96744,96827,96736,96758,87.539,91.000000,up,"[[39, 39, 98273], [69, 69, 98640], [104, 104, ...","[[45, 45, 98458], [96, 96, 98528], [126, 126, ...",0
1,2024-12-05 03:43:00,96710,96721,96710,96721,4.998,69.500000,up,"[[39, 39, 98273], [69, 69, 98640], [104, 104, ...","[[45, 45, 98458], [96, 96, 98528], [126, 126, ...",0
2,2024-12-05 03:44:00,96870,96871,96850,96854,12.117,96.333333,up,"[[39, 39, 98273], [69, 69, 98640], [104, 104, ...","[[45, 45, 98458], [96, 96, 98528], [126, 126, ...",0
3,2024-12-05 03:45:00,96832,96833,96821,96832,2.831,80.500000,up,"[[39, 39, 98273], [69, 69, 98640], [104, 104, ...","[[45, 45, 98458], [96, 96, 98528], [126, 126, ...",0
4,2024-12-05 03:46:00,96731,96731,96719,96720,2.721,87.000000,up,"[[39, 39, 98273], [69, 69, 98640], [104, 104, ...","[[45, 45, 98458], [96, 96, 98528], [126, 126, ...",0
...,...,...,...,...,...,...,...,...,...,...,...
139,2024-12-05 06:09:00,98037,98080,98033,98034,2.396,107.486624,up,"[[39, 39, 98273], [69, 69, 98640], [104, 104, ...","[[45, 45, 98458], [96, 96, 98528], [126, 126, ...",0
140,2024-12-05 06:10:00,98015,98015,98014,98014,0.171,101.237579,up,"[[39, 39, 98273], [69, 69, 98640], [104, 104, ...","[[45, 45, 98458], [96, 96, 98528], [126, 126, ...",0
141,2024-12-05 06:11:00,97943,97980,97925,97925,3.876,100.363467,up,"[[39, 39, 98273], [69, 69, 98640], [104, 104, ...","[[45, 45, 98458], [96, 96, 98528], [126, 126, ...",0
142,2024-12-05 06:12:00,97945,97946,97899,97946,2.482,96.551790,up,"[[39, 39, 98273], [69, 69, 98640], [104, 104, ...","[[45, 45, 98458], [96, 96, 98528], [126, 126, ...",0
